## Notebook for learning the AutoGen framework

https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/index.html

In [1]:
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
# Define a model client. You can use other model client that implements
# the `ChatCompletionClient` interface.
model_client_small = OpenAIChatCompletionClient(
    model="gpt-4.1-mini",
)
model_client_large = OpenAIChatCompletionClient(
    model="gpt-4.1",
)

### Code review example in a team of 2 agents

In [4]:
# Create the primary agent.
junior_coder = AssistantAgent(
    "coder",
    model_client=model_client_small,
    # system_message="You are a junior programmer just starting with Python and JS. You need to write code as per requirements",
    system_message="You are a junior programmer just starting with Python and JS.",
)
# Create the critic agent.
tech_lead = AssistantAgent(
    "tech_lead",
    model_client=model_client_large,
    system_message="Provide constructive feedback on the code. Respond with 'APPROVED' when your feedbacks are addressed.",
)

In [5]:
# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVED")

# Create a team with the primary and critic agents.
team = RoundRobinGroupChat([junior_coder, tech_lead], termination_condition=text_termination)

In [7]:
team_task = "Write python code using numpy to return inverse of a matrix"
await team.reset()  # Reset the team for a new task.
await Console(team.run_stream(task=team_task))  # Stream the messages to the console.

---------- TextMessage (user) ----------
Write python code using numpy to return inverse of a matrix
---------- TextMessage (coder) ----------
Certainly! Here's a simple Python code using NumPy to return the inverse of a matrix:

```python
import numpy as np

def inverse_matrix(matrix):
    try:
        inv_matrix = np.linalg.inv(matrix)
        return inv_matrix
    except np.linalg.LinAlgError:
        return "Matrix is singular and cannot be inverted."

# Example usage:
matrix = np.array([[1, 2], [3, 4]])
print(inverse_matrix(matrix))
```

This function tries to compute the inverse of the input matrix and handles the case where the matrix is singular (non-invertible).
---------- TextMessage (tech_lead) ----------
**Feedback:**

1. Good use of try-except for handling singular matrices.
2. Consider adding input validation to ensure `matrix` is a 2D NumPy array and is square. 
3. The message for singular matrices is returned as a string, which may break consistency with numeric return 

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write python code using numpy to return inverse of a matrix', type='TextMessage'), TextMessage(source='coder', models_usage=RequestUsage(prompt_tokens=35, completion_tokens=123), metadata={}, content='Certainly! Here\'s a simple Python code using NumPy to return the inverse of a matrix:\n\n```python\nimport numpy as np\n\ndef inverse_matrix(matrix):\n    try:\n        inv_matrix = np.linalg.inv(matrix)\n        return inv_matrix\n    except np.linalg.LinAlgError:\n        return "Matrix is singular and cannot be inverted."\n\n# Example usage:\nmatrix = np.array([[1, 2], [3, 4]])\nprint(inverse_matrix(matrix))\n```\n\nThis function tries to compute the inverse of the input matrix and handles the case where the matrix is singular (non-invertible).', type='TextMessage'), TextMessage(source='tech_lead', models_usage=RequestUsage(prompt_tokens=172, completion_tokens=236), metadata={}, content='**Feedbac